# CSCA 5642 - Deep Learning Final Project


### Project Topic

The topic of this project is Reinforcement Learning but also Deep Learning as this project will use Deep Q Learning (DQN) which involves multilayer Neural Networks like we have been using throughout the CSCA5642 class. I was interested in Reinforcement Learning and was hoping we would go over this in class, and since we did it, I am doing my final project on RL.


## Problem Description 

Source for Project Data:
https://www.gymlibrary.dev/environments/classic_control/cart_pole/

The Machine Learning problem provided at the above link is a simple Reinforcement Learning Game. In this game there is a cart that is defined to moving in one dimension, left or right. On top of the cart there is a balanced pole and the pole will fall over if not corrected. The cart may move either left or right to keep the pole balanced on top of the cart.

![Problem Description](images/problem_description.png)

### Data Description


![Boundaries for data](images/problem_boundaries.png)


In [ ]:
import os
import gym
import numpy as np
import random
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import pandas as pd
from matplotlib import pyplot as plt

## Exploratory Data Analysis

TOOD - Take a deeper look at the data

## Model Building

Lets start by building a basic model, we will call this model 1

##